In [1]:
import os
import numpy as np
import torch
import torchvision
import pycocotools
import pandas as pd
from PIL import Image
from torch.utils.data import Subset, DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch.nn as nn
from torchvision.transforms import v2
import matplotlib.pyplot as plt
from collections import Counter
import xmltodict
import statistics

In [2]:
torch.cuda.set_device(3)

In [3]:
def load_model_from_file(num_classes, filename, flag=True):   
    # Load an object detection model pretrained on MS COCO.
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=flag, weights_backbone=flag)
    # Load the fasterrcnn resnet50.
    chk = torch.load(filename)
    # Load state dictionary.
    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new on
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    # return model.
    model.load_state_dict(chk)
    
    return model

In [4]:
num_classes = 4
# model_filename = 'trained_model_faster_rcnn_epoch_370_loss_60.pth'
# model_filename = 'trained_model_faster_rcnn_epoch_10_loss_58.pth' # 8.5
# model_filename = 'trained_model_faster_rcnn_epoch_40_loss_61.pth' # 9.06
# model_filename = 'trained_model_faster_rcnn_epoch_60_loss_62.pth' # 9.69
# model_filename = 'trained_model_faster_rcnn_epoch_70_loss_61.pth' # 9.59
# model_filename = 'trained_model_faster_rcnn_epoch_90_loss_63.pth' # 9.61
# model_filename = 'trained_model_faster_rcnn_epoch_100_loss_61.pth' # 9.5
# model_filename = 'trained_model_faster_rcnn_epoch_120_loss_63.pth' # 9.5
# model_filename = 'trained_model_faster_rcnn_epoch_10_loss_121.pth' # 
# model_filename = 'trained_model_faster_rcnn_epoch_48_loss_69.pth' # 8.5
# model_filename = 'trained_model_faster_rcnn_epoch_245_loss_68_Final.pth'

model_filename = 'trained_model_faster_rcnn_epoch_464_loss_24.pth'

loaded_model = load_model_from_file(num_classes, model_filename)

/opt/omniai/software/Miniconda/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/opt/omniai/software/Miniconda/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights_backbone' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights_backbone=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights_backbone=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FileNotFoundError: [Errno 2] No such file or directory: 'trained_model_faster_rcnn_epoch_464_loss_24.pth'

In [ ]:
class MaskedTestDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        self.annotations_list = sorted(os.listdir(os.path.join(root, 'annotations_val')))
        self.imgs = sorted(glob.glob(os.path.join(root,'val' , '*.png')))

    def __getitem__(self, indx):
        
        # load annotation
        filename, boxes, labels = parse_xml(os.path.join(self.root, 'annotations_val', self.annotations_list[indx]))
        
        
        # load image
        img_path = os.path.join(self.root, 'val', filename)
        
        img = Image.open(img_path).convert('RGB')
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        num_objs = boxes.shape[0]
        
        #classes
        labels = torch.tensor(labels, dtype=torch.int64)
        image_id = torch.tensor([indx])
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = image_id
        target['area'] = area
        target['iscrowd'] =iscrowd
        
        if self.transforms is not None:
            img, target = self.transforms(img, target)
        return img, target
    
    def __len__(self):
        
        return len(self.imgs)

In [ ]:
# Dataset
import os, glob
from PIL import Image
from torch.utils.data import Dataset
class MaskedFaceTestDataset(Dataset):
    def __init__(self, root, transform=None):
        super(MaskedFaceTestDataset, self).__init__()
        self.imgs = sorted(glob.glob(os.path.join(root, '*.png')))
        self.transform = transform

    def __getitem__(self, index):
        img_path = self.imgs[index]
        img = Image.open(img_path).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        return img

    def __len__(self):
        return len(self.imgs)

In [ ]:
'''
def get_transform(train):
    #Transform of the images.
    transform_list= []
    transform_list.append(v2.PILToTensor())
    transform_list.append(v2.ConvertImageDtype(torch.float))
    
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        # This configuration best for training.
        transform_list.append(v2.RandomHorizontalFlip(0.5))
        # Random affine not working well at lower epochs.
        transform_list.append(v2.RandomAffine(5))   
        # Image resize transformation does not work well for this training.
        transform_list.append(v2.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5)))
        
    transform_object = v2.Compose(transform_list)
    
    return transform_object
 '''

In [ ]:
def get_transform(train):
    #Transform of the images.
    transform_list= []
    
    transform_list.append(v2.PILToTensor())
    transform_list.append(v2.ConvertImageDtype(torch.float))
    
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        # This configuration best for training.
        # transform_list.append(v2.Resize((256,256), antialias=None))
        # transform_list.append(v2.RandomResizedCrop(size=(224, 224), antialias=True))
        # transform_list.append(v2.ColorJitter())
        transform_list.append(v2.RandomHorizontalFlip(0.5))
        # Random affine not working well at lower epochs.
        # transform_list.append(v2.RandomAffine(15))   
        # Image resize transformation does not work well for this training.
        #transform_list.append(v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]))
    #else:
        # transform_list.append(v2.Resize((256,256), antialias=None))
        # transform_list.append(v2.RandomResizedCrop(size=(224, 224), antialias=True))
        #transform_list.append(v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]))
    
    transform_object = v2.Compose(transform_list)
    
    return transform_object

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
val_dataset = MaskedTestDataset(root='MaskedFace', transforms=get_transform(train=False))
print(len(val_dataset))
# data loader
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=True, num_workers=12, collate_fn=collate_fn)

In [ ]:
test_dataset = MaskedFaceTestDataset(root='MaskedFaceTest', transform=get_transform(train=False))
#test_dataset = MaskedFaceTestDataset(root='MaskedFaceTest')
print(len(test_dataset))
# data loader
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=True, num_workers=12, collate_fn=collate_fn)

In [ ]:
infer_model = loaded_model.cpu()
infer_model.eval()
print("Evaluation Mode of Model:-")

In [ ]:
class_str2num = {'with_mask': 1,'without_mask': 2,'mask_weared_incorrect': 3}
class_num2str = {v: k for k, v in class_str2num.items()}

In [ ]:
import xml.etree.ElementTree as ET
def parse_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    bboxes = []
    labels = []
    filename = root.find('filename').text
    for boxes in root.iter('object'):
        ymin, xmin, ymax, xmax = None, None, None, None
        ymin = int(boxes.find("bndbox/ymin").text)
        xmin = int(boxes.find("bndbox/xmin").text)
        ymax = int(boxes.find("bndbox/ymax").text)
        xmax = int(boxes.find("bndbox/xmax").text)
        box = [xmin, ymin, xmax, ymax]
        bboxes.append(box)
        labels.append(int(class_str2num[boxes.find("name").text]))
    return filename, bboxes, labels

In [ ]:
%%time
from collections import Counter
print("Class numbers")
print(class_num2str)

def calculate_MAPE(test_dataset, confidence_score, model):
    
    class_score = 0
    class_score_ind=[]
    
    for i in range(len(test_dataset)):
        
        img, _ = test_dataset[i]
        
        with torch.no_grad():
            # Prediction for the particular image.
            prediction = model([img])
        
        groundtruth_labels = test_dataset[i][1]['labels'].numpy()
        confident_prediction_labels=[]
        
        # For all the predicted boxes check the labels and confidence scores.
        for element in range(len(prediction[0]['boxes'])):

            #print("Prediction Label of each box:-")
            ele_pred_label = prediction[0]['labels'][element].cpu().item()
            #print(ele_pred_label)
            
            score = np.round(prediction[0]['scores'][element].cpu().numpy(), decimals= 4)
            # if the score is greater than confidence score.
            if score > confidence_score:
               # print(score)
               # if  the score is greater than the confidence score
               confident_prediction_labels.append(ele_pred_label)
        
        #print("The confident prediction labels:-")
        #print(confident_prediction_labels)
        C_grd = Counter(groundtruth_labels)
        #print(C_grd)
        C_pred = Counter(confident_prediction_labels)   
        #print(C_pred)
        
        # Classes in the Object detection:-
        # print(class_num2str)
        class_score=0
        # Class keys.
        for k in class_num2str.keys():
            # print("Class Labels:-")
            # print(k)
            At = C_grd[k]
            Pt = C_pred[k] 
            class_score += abs(At-Pt)/max(At,1)
        
        # Find the average class score of all the classes.
        class_score_ind.append(class_score/(len(class_num2str.keys())+1))
    
    sum_scores = sum(class_score_ind)
    #print("Scores_sum")
    #print(sum_scores)
    #print("Class Scores:-")
    #print(class_score)  
    #print("Class_score_List:-")
    #print(class_score_ind)
    #print("length of class scores:-")
    #print(len(class_score_ind))
    #print("length of test dataset")
    #print(len(test_dataset))
    Avg_MAPE = (sum_scores/len(test_dataset)) * 100
    print("Avg MAPE Score on the dataset")
    print(Avg_MAPE)
    
confidence_score = 0.7
calculate_MAPE(val_dataset, confidence_score, infer_model)

In [ ]:
# Count masked faces
def count_masks_val(input_dataset, model, confidence_score=0.5):
    
    len_dataset = len(input_dataset)
    
    inference_results_matrix = np.zeros((len_dataset,3))

    for i in range(len(input_dataset)):
        img, _ = input_dataset[i]
        
        with torch.no_grad():
            prediction = model([img])
    
        for element in range(len(prediction[0]['boxes'])):
            label = prediction[0]['labels'][element].cpu().item()
            score = np.round(prediction[0]['scores'][element].cpu().numpy(), decimals= 4)
            if score > confidence_score:
               inference_results_matrix[i][label-1] += 1 
            
    return inference_results_matrix

In [ ]:
# Count masked faces
def count_masks_test(input_dataset, model, confidence_score=0.5):
    
    len_dataset = len(input_dataset)
    
    inference_results_matrix = np.zeros((len_dataset,3))

    for i in range(len(input_dataset)):
        img = input_dataset[i]
        
        with torch.no_grad():
            prediction = model([img])
    
        for element in range(len(prediction[0]['boxes'])):
            label = prediction[0]['labels'][element].cpu().item()
            score = np.round(prediction[0]['scores'][element].cpu().numpy(), decimals= 4)
            if score > confidence_score:
               inference_results_matrix[i][label-1] += 1 
            
    return inference_results_matrix

In [ ]:
%%time
confidence_score=0.6
inf_results = count_masks_val(val_dataset, infer_model, confidence_score)
print(inf_results.shape)

In [ ]:
%%time
confidence_score=0.7
inf_results1 = count_masks_test(test_dataset, infer_model, confidence_score)
print(inf_results1.shape)
print(inf_results1)

In [ ]:
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10.0, 10.0)
plt.rcParams['figure.dpi'] = 72

# Iterate the test dataset.
for i in range(len(test_dataset)):
    img = test_dataset[i]
    
    #put the model in evaluation mode
    with torch.no_grad():
        prediction = infer_model([img])    
    image = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
    draw = ImageDraw.Draw(image)
    
    # Plot each prediction boxes.
    for element in range(len(prediction[0]['boxes'])):
        box = prediction[0]['boxes'][element].cpu().numpy()
        label = prediction[0]['labels'][element].cpu().item()
        score = np.round(prediction[0]['scores'][element].cpu().numpy(), decimals= 4)
        if score > 0.7:
            draw.rectangle([(box[0], box[1]), (box[2], box[3])], outline ='blue', width =3)
            draw.text((box[0], box[1]), text = class_num2str[label] + ', ' + str(score))
    plt.imshow(image)
    plt.show()

In [ ]:
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10.0, 10.0)
plt.rcParams['figure.dpi'] = 72

for i in range(len(val_dataset)):
    img, _ = val_dataset[i]
    label_boxes = np.array(val_dataset[i][1]['boxes'])
    #put the model in evaluation mode
    with torch.no_grad():
        prediction = infer_model([img])
    image = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
    draw = ImageDraw.Draw(image)
    # draw groundtruth
    for elem in range(len(label_boxes)):
        draw.rectangle([(label_boxes[elem][0], label_boxes[elem][1]), (label_boxes[elem][2], label_boxes[elem][3])], outline ="yellow", width =3)
    for element in range(len(prediction[0]['boxes'])):
        box = prediction[0]['boxes'][element].cpu().numpy()
        label = prediction[0]['labels'][element].cpu().item()
        score = np.round(prediction[0]['scores'][element].cpu().numpy(), decimals= 4)
        if score > 0.6:
            draw.rectangle([(box[0], box[1]), (box[2], box[3])], outline ='blue', width =3)
            draw.text((box[0], box[1]), text = class_num2str[label] + ', ' + str(score))
    plt.imshow(image)
    plt.show()